In [1]:
import yfinance as yf

def get_fx_rates(symbols):
    data = yf.download(symbols, period="5d", interval="1d", group_by='ticker', progress=False)
    result = {}
    for symbol in symbols:
        try:
            close_series = data[symbol]["Close"].dropna()
            result[symbol] = close_series.iloc[-1] if not close_series.empty else None
        except:
            result[symbol] = None
    return result

def get_crude_prices():
    symbols = {"WTI": "CL=F", "Brent": "BZ=F"}
    prices = get_fx_rates(list(symbols.values()))
    return {name: prices[sym] for name, sym in symbols.items()}

def get_gold_prices():
    symbols = {"Gold": "GC=F"}
    prices = get_fx_rates(list(symbols.values()))
    return {name: prices[sym] for name, sym in symbols.items()}

def get_us_10y_rate():
    try:
        data = yf.download("^TNX", period="5d", interval="1d", progress=False)
        tnx_value = data["Close"].iloc[-1]
        return float(tnx_value)
    except:
        return None

def get_global_indices():
    import yfinance as yf

    symbols = {
        "S&P 500": "^GSPC", "Dow Jones": "^DJI", "Nasdaq": "^IXIC", "IPC México": "^MXX",
        "Bovespa Brasil": "^BVSP", "FTSE 100 UK": "^FTSE", "DAX Alemania": "^GDAXI",
        "CAC 40 Francia": "^FCHI", "Nikkei Japón": "^N225", "Hang Seng": "^HSI",
        "Shanghai Comp.": "000001.SS", "Kospi Corea": "^KS11", "Sensex India": "^BSESN",
        "ASX 200 Australia": "^AXJO"
    }

    data = yf.download(list(symbols.values()), period="5d", interval="1d", group_by='ticker', progress=False)
    result = {}
    for name, sym in symbols.items():
        try:
            close_series = data[sym]["Close"].dropna()
            result[name] = close_series.iloc[-1] if not close_series.empty else None
        except Exception:
            result[name] = None
    return result



In [2]:
from datetime import datetime

# Ejecución de scraping
symbols = [
    "BRL=X", "CLP=X", "COP=X", "KRW=X", "ZAR=X", "INR=X", "MXN=X", "PEN=X",
    "PLN=X", "CZK=X", "RUB=X", "THB=X", "TRY=X", "HUF=X", "ILS=X",
    "DX-Y.NYB", "ARS=X", "CNY=X", "IDR=X", "MYR=X"
]
fx_data = get_fx_rates(symbols)
crude = get_crude_prices()
gold = get_gold_prices()
us_rate = get_us_10y_rate()
indices = get_global_indices()


datos_diarios = {
    "fecha": datetime.today().strftime("%d/%m/%y"),
    "fx": fx_data,
    "crudo": get_crude_prices(),
    "oro": get_gold_prices(),
    "tasas": {"USY": get_us_10y_rate()},
    "indices": get_global_indices()
}


YF.download() has changed argument auto_adjust default to True


/var/folders/pk/5vtlxqtx3l11lzrj4g1k6sl80000gn/T/ipykernel_35679/2036884082.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(tnx_value)


/var/folders/pk/5vtlxqtx3l11lzrj4g1k6sl80000gn/T/ipykernel_35679/2036884082.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(tnx_value)


In [3]:
from datetime import datetime
import os
import json

# Define ubicación actual
#src_path = "./src"
#os.chdir(src_path)

# Asegurar carpeta
base_dir = "../data"
os.makedirs(base_dir, exist_ok=True)

# Guardar con fecha del día (scraped_data_ddmmyy.json)
fecha_hoy = datetime.today().strftime("%d%m%y")
json_diario = f"scraped_data_{fecha_hoy}.json"
json_path = os.path.join(base_dir, json_diario)

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(datos_diarios, f, indent=2, ensure_ascii=False)

print(f"✅ Datos guardados como snapshot diario: {json_path}")


✅ Datos guardados como snapshot diario: ../data/scraped_data_180525.json


In [4]:
# Calcular fecha anterior con lógica de fin de semana
import calendar
from datetime import datetime, timedelta


def obtener_fecha_anterior(fecha_actual):
    dia_semana = fecha_actual.weekday()  # lunes=0 ... domingo=6
    if dia_semana == 0:  # lunes
        delta = 3
    elif dia_semana == 6:  # domingo
        delta = 2
    else:
        delta = 1
    return fecha_actual - timedelta(days=delta)

# Leer archivo anterior si existe
fecha_actual_dt = datetime.today()
fecha_ayer_dt = obtener_fecha_anterior(fecha_actual_dt)
archivo_ayer = f"scraped_data_{fecha_ayer_dt.strftime('%d%m%y')}.json"
ruta_ayer = os.path.join(base_dir, archivo_ayer)

indices_anteriores = {}
if os.path.exists(ruta_ayer):
    with open(ruta_ayer, "r", encoding="utf-8") as f:
        datos_ayer = json.load(f)
        indices_anteriores = datos_ayer.get("indices", {})

# Calcular variaciones
indices_actuales = datos_diarios.get("indices", {})
indices_variacion = {}
for nombre, valor_actual in indices_actuales.items():
    valor_anterior = indices_anteriores.get(nombre)
    if valor_actual is not None and valor_anterior is not None:
        try:
            variacion = ((valor_actual - valor_anterior) / valor_anterior) * 100
            indices_variacion[nombre] = round(variacion, 2)
        except ZeroDivisionError:
            indices_variacion[nombre] = None
    else:
        indices_variacion[nombre] = None


In [5]:
# Guardar json con alias scraped_data.json incluyendo variaciones
datos_diarios["indices_variacion"] = indices_variacion

alias_path = os.path.join(base_dir, "scraped_data.json")
with open(alias_path, "w", encoding="utf-8") as f:
    json.dump(datos_diarios, f, indent=2, ensure_ascii=False)

print("✅ Variaciones calculadas y guardadas en scraped_data.json")


✅ Variaciones calculadas y guardadas en scraped_data.json
